# Introduction to embeddings with the OpenAI API

In [ ]:
import sys
import os

# Use current working directory and go one level up
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(parent_dir)

# Now you can import your config
from config import api_key

from openai import OpenAI

## Chapter 1 - What are embeddings?

### Section 1.1 - The wonderful world of embeddings!

#### Creating embeddings
In this exercise, you'll create your very first embeddings using the OpenAI API. Normally, to interact with the OpenAI API, you would need an OpenAI API key, and creating embeddings would incur a cost. However, you do not need to create or provide an API key in this course.

The OPENAI_API_TOKEN placeholder has been provided in the code, which will send valid requests for the exercises in this course. If, at any point in the course, you hit a `RateLimitError`, pause for a moment and try again.

The `OpenAI` class from the `openai` library will be imported for you throughout the course, and after this exercise, the `client` will be created for you.

In [ ]:
# Create an OpenAI client
client = OpenAI(api_key=api_key)

# Create a request to obtain embeddings
response = client.embeddings.create(
    model="text-embedding-3-small",
    input="this is a text to embed"
)

# Convert the response into a dictionary
response_dict = response.model_dump()
print(response_dict)

#### Digging into the embeddings response

You've been able to successfully use the OpenAI Embeddings endpoint to embed text data, and in this exercise, you'll finish this off by extracting information from the API's response.

You've been provided with a response from the Embeddings API, which has already been converted into a dictionary and stored as `response_dict`. You'll need to extract the desired information from this dictionary. This `response_dict` has been printed for you, so you can view its contents and structure.

Recall that the response is structured like a nested Python dictionary, and it can be accessed in much the same way.

In [ ]:
response_dict.keys()

In [ ]:
response_dict['usage']['total_tokens']

In [ ]:
response_dict['data'][0]['embedding'][:10]

### Section 1.2 - Investigating the vector space

#### Embedding product descriptions

**remark** - this is a rather complex exercise. It can help to break it down in peaces to understand the details.

You've been provided with a list of dictionaries called `products`, which contains product information for different products sold by an online retailer. It's your job to embed the `'short_description'` for each product to enable semantic search for the retailer's website.

Here's a preview of the `products` list of dictionaries:
```
products = [
    {
        "title": "Smartphone X1",
        "short_description": "The latest flagship smartphone with AI-powered features and 5G connectivity.",
        "price": 799.99,
        "category": "Electronics",
        "features": [
            "6.5-inch AMOLED display",
            "Quad-camera system with 48MP main sensor",
            "Face recognition and fingerprint sensor",
            "Fast wireless charging"
        ]
    },
    ...
]```

An OpenAI client has already been created as assigned to client.

In [ ]:
products = [{'title': 'Smartphone X1',
  'short_description': 'The latest flagship smartphone with AI-powered features and 5G connectivity.',
  'price': 799.99,
  'category': 'Electronics',
  'features': ['6.5-inch AMOLED display',
   'Quad-camera system with 48MP main sensor',
   'Face recognition and fingerprint sensor',
   'Fast wireless charging']},
 {'title': 'Luxury Diamond Necklace',
  'short_description': 'Elegant necklace featuring genuine diamonds, perfect for special occasions.',
  'price': 1499.99,
  'category': 'Beauty',
  'features': ['18k white gold chain',
   '0.5 carat diamond pendant',
   'Adjustable chain length',
   'Gift box included']},
 {'title': 'RC Racing Car',
  'short_description': 'High-speed remote-controlled racing car for adrenaline-packed fun.',
  'price': 89.99,
  'category': 'Toys',
  'features': ['Top speed of 30 mph',
   'Responsive remote control',
   'Rechargeable battery',
   'Durable construction']},
 {'title': 'Ultra HD 4K TV',
  'short_description': 'Immerse yourself in stunning visuals with this 65-inch 4K TV.',
  'price': 1299.99,
  'category': 'Electronics',
  'features': ['65-inch 4K UHD display',
   'Dolby Vision and HDR10+ support',
   'Smart TV with streaming apps',
   'Voice remote included']},
 {'title': 'Glowing Skin Serum',
  'short_description': 'Revitalize your skin with this nourishing serum for a radiant glow.',
  'price': 39.99,
  'category': 'Beauty',
  'features': ['Hyaluronic acid and vitamin C',
   'Hydrates and reduces fine lines',
   'Suitable for all skin types',
   'Cruelty-free']},
 {'title': 'LEGO Space Shuttle',
  'short_description': 'Build your own space adventure with this LEGO space shuttle set.',
  'price': 49.99,
  'category': 'Toys',
  'features': ['359 pieces for creative building',
   'Astronaut minifigure included',
   'Compatible with other LEGO sets',
   'For ages 7+']},
 {'title': 'Wireless Noise-Canceling Headphones',
  'short_description': 'Enjoy immersive audio and block out distractions with these headphones.',
  'price': 199.99,
  'category': 'Electronics',
  'features': ['Active noise cancellation',
   'Bluetooth 5.0 connectivity',
   'Long-lasting battery life',
   'Foldable design for portability']},
 {'title': 'Luxury Perfume Gift Set',
  'short_description': 'Indulge in a collection of premium fragrances with this gift set.',
  'price': 129.99,
  'category': 'Beauty',
  'features': ['Five unique scents',
   'Elegant packaging',
   'Perfect gift for fragrance enthusiasts',
   'Variety of fragrance notes']},
 {'title': 'Remote-Controlled Drone',
  'short_description': 'Take to the skies and capture stunning aerial footage with this drone.',
  'price': 299.99,
  'category': 'Electronics',
  'features': ['4K camera with gimbal stabilization',
   'GPS-assisted flight',
   'Remote control with smartphone app',
   'Return-to-home function']},
 {'title': 'Luxurious Spa Gift Basket',
  'short_description': 'Pamper yourself or a loved one with this spa gift basket full of relaxation goodies.',
  'price': 79.99,
  'category': 'Beauty',
  'features': ['Bath bombs, body lotion, and more',
   'Aromatherapy candles',
   'Reusable wicker basket',
   'Great for self-care']},
 {'title': 'Robot Building Kit',
  'short_description': 'Learn robotics and coding with this educational robot building kit.',
  'price': 59.99,
  'category': 'Toys',
  'features': ['Build and program your own robot',
   'STEM learning tool',
   'Compatible with Scratch and Python',
   'Ideal for young inventors']},
 {'title': 'High-Performance Gaming Laptop',
  'short_description': 'Dominate the gaming world with this powerful gaming laptop.',
  'price': 1499.99,
  'category': 'Electronics',
  'features': ['Intel Core i7 processor',
   'NVIDIA RTX graphics',
   '144Hz refresh rate display',
   'RGB backlit keyboard']},
 {'title': 'Natural Mineral Makeup Set',
  'short_description': 'Enhance your beauty with this mineral makeup set for a flawless look.',
  'price': 34.99,
  'category': 'Beauty',
  'features': ['Mineral foundation and eyeshadows',
   'Non-comedogenic and paraben-free',
   'Cruelty-free and vegan',
   'Includes makeup brushes']},
 {'title': 'Interactive Robot Pet',
  'short_description': 'Adopt your own robot pet that responds to your voice and touch.',
  'price': 79.99,
  'category': 'Toys',
  'features': ['Realistic pet behaviors',
   'Voice recognition and touch sensors',
   'Teaches responsibility and empathy',
   'Rechargeable battery']},
 {'title': 'Smart Thermostat',
  'short_description': "Control your home's temperature and save energy with this smart thermostat.",
  'price': 129.99,
  'category': 'Electronics',
  'features': ['Wi-Fi connectivity',
   'Energy-saving features',
   'Compatible with voice assistants',
   'Easy installation']},
 {'title': 'Designer Makeup Brush Set',
  'short_description': 'Upgrade your makeup routine with this premium designer brush set.',
  'price': 59.99,
  'category': 'Beauty',
  'features': ['High-quality synthetic bristles',
   'Chic designer brush handles',
   'Complete set for all makeup needs',
   'Includes stylish carrying case']},
 {'title': 'Remote-Controlled Dinosaur Toy',
  'short_description': 'Roar into action with this remote-controlled dinosaur toy with lifelike movements.',
  'price': 49.99,
  'category': 'Toys',
  'features': ['Realistic dinosaur sound effects',
   'Walks and roars like a real dinosaur',
   'Remote control included',
   'Educational and entertaining']},
 {'title': 'Wireless Charging Dock',
  'short_description': 'Charge your devices conveniently with this sleek wireless charging dock.',
  'price': 39.99,
  'category': 'Electronics',
  'features': ['Qi wireless charging technology',
   'Supports multiple devices',
   'LED charging indicators',
   'Compact and stylish design']},
 {'title': 'Luxury Skincare Set',
  'short_description': 'Elevate your skincare routine with this luxurious skincare set.',
  'price': 179.99,
  'category': 'Beauty',
  'features': ['Premium anti-aging ingredients',
   'Hydrating and rejuvenating formulas',
   'Complete skincare regimen',
   'Elegant packaging']}]

In [ ]:
# Extract a list of product short descriptions from products
product_descriptions = [product['short_description'] for product in products]

# Create embeddings for each product description
response = client.embeddings.create(
    model="text-embedding-3-small",
    input=product_descriptions)

response_dict = response.model_dump()

# Extract the embeddings from response_dict and store in products
for i, product in enumerate(products):
    product['embedding'] = response_dict['data'][i]['embedding']   
     
# print(products[0].items())

#### Visualizing the embedded descriptions

Now that you've created embeddings from the product descriptions, it's time to explore them! You'll use t-SNE to reduce the number of dimensions in the embeddings data from 1,536 to two, which will make the data much easier to visualize.

You'll start with the products list of dictionaries you worked with in the last exercise, containing product information and the embeddings you created from the 'short_description'. As a reminder, here's a preview of products:
```
products = [
    {
        "title": "Smartphone X1",
        "short_description": "The latest flagship smartphone with AI-powered features and 5G connectivity.",
        "price": 799.99,
        "category": "Electronics",
        "features": [
            "6.5-inch AMOLED display",
            "Quad-camera system with 48MP main sensor",
            "Face recognition and fingerprint sensor",
            "Fast wireless charging"
        ],
        "embedding": [-0.014650369994342327, ..., 0.008677126839756966]
    },
    ...
]```

`matplotlib.pyplot` and `numpy` have been imported as `plt` and `np`, respectively.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.manifold import TSNE

In [ ]:
# Create reviews and embeddings lists using list comprehensions
categories = [product['category'] for product in products]
embeddings = [product['embedding'] for product in products]

# Reduce the number of embeddings dimensions to two using t-SNE
tsne = TSNE(n_components=2, perplexity=5)
embeddings_2d = tsne.fit_transform(np.array(embeddings))

# Create a scatter plot from embeddings_2d
plt.scatter(embeddings_2d[:,0], embeddings_2d[:,1])

for i, category in enumerate(categories):
    plt.annotate(category, (embeddings_2d[i, 0], embeddings_2d[i, 1]))

plt.show()

### Section 1.3 - Text similarity

#### Computing cosine distances
To identify the most semantically similar texts, you need to apply a distance metric. A popular choice is the cosine distance.

In this exercise, you've been provided with four vectors: `A`, `B`, `C`, and `D`. It's your task to find out which vector is most similar to `A` using cosine distance.

Which vector is the most similar to `A`?

`distance` has already been imported from scipy.spatial.

In [ ]:
from scipy.spatial import distance

In [ ]:
A = [3, 1, 4]
B = [1, 5, 9]
C = [2, 6, 5]
D = [3, 5, 9]

In [ ]:
print(distance.cosine(A,B))
print(distance.cosine(A,C))
print(distance.cosine(A,D))

#### More repeatable embeddings

As you continue to work with embeddings, you'll find yourself making repeated calls to OpenAI's embedding model. To make these calls in a more repeatable and modular way, it would be better to define a custom function called `create_embeddings()` that would output embeddings for any number of text inputs. In this exercise, you'll do just that!

In [ ]:
short_description = 'The latest flagship smartphone with AI-powered features and 5G connectivity.'
list_of_descriptions = ['Charge your devices conveniently with this sleek wireless charging dock.',
 'Elevate your skincare routine with this luxurious skincare set.']


In [ ]:
# Define a create_embeddings function
def create_embeddings(texts):
  response = client.embeddings.create(
    model="text-embedding-3-small",
    input=texts
  )
  response_dict = response.model_dump()

  return [data['embedding'] for data in response_dict['data']]

# Embed short_description and print
print(create_embeddings(short_description)[0])

# Embed list_of_descriptions and print
print(create_embeddings(list_of_descriptions))

#### Finding the most similar product
Being able to compute similarity between embeddings is a key step within embeddings applications. In this exercise, you'll return to the products list of dictionaries that you worked with previously, which contains the embedded short descriptions you also created earlier.

You'll compare a piece of text to these embedded descriptions to identify the most similar description.

`numpy` has been imported as `np`, and `distance` is available from `scipy.spatial`. A `create_embeddings()` function has already been defined for you and is available to use for creating embeddings from a single input.

In [ ]:
from scipy.spatial import distance
import numpy as np

In [ ]:
# Embed the search text
search_text = "soap"
search_embedding = create_embeddings(search_text)[0]

distances = []
for product in products:
  # Compute the cosine distance for each product description
  dist = distance.cosine(search_embedding, product['embedding'])
  distances.append(dist)

# Find and print the most similar product short_description    
min_dist_ind = np.argmin(distances)
print(products[min_dist_ind]['short_description'])

## Chapter 2 - Embedding for AI Applications

### Section 2.1 - Semantic search and enriched embeddings

#### Enriching embeddings\

Previously, when you embedded product information, you were limited to only embedding the product 'short_description', which captured some, but not all of the relevant product information available. In this exercise, you'll embed `'title'`, `'short_description'`, `'category'`, and `'features'` to capture much more information.

Here's a reminder of the products list of dictionaries:

When combining the features into a single string, it should have the following structure:

In [ ]:
# Define a function to combine the relevant features into a single string
def create_product_text(product):
  return f"""Title: {product['title']}
Description: {product['short_description']}
Category: {product['category']},
Features: {", ".join(product['features'])}
"""

# Combine the features for each product
product_texts = [create_product_text(product) for product in products]

# Create the embeddings from product_texts
product_embeddings = create_embeddings(product_texts)

#### Sorting by similarity

Now that you've embedded all of your features, the next step is to compute the similarities. In this exercise, you'll define a function called `find_n_closest()`, which computes the cosine distances between a query vector and a list of embeddings and returns the `n` smallest distances and their indexes.

In the next exercise, you'll use this function to enable your semantic product search application.

`distance` has been imported from `scipy.spatial`.

In [ ]:
import numpy as np
from scipy.spatial import distance

In [ ]:
def find_n_closest(query_vector, embeddings, n=3):
  distances = []
  for index, embedding in enumerate(embeddings):
    # Calculate the cosine distance between the query vector and embedding
    dist = distance.cosine(query_vector, embedding)
    # Append the distance and index to distances
    distances.append({"distance": dist, "index": index})
  # Sort distances by the distance key
  distances_sorted = sorted(distances, key=lambda x: x['distance'])
  # Return the first n elements in distances_sorted
  return distances_sorted[0:n]

#### Semantic search for products

Time to put your `find_n_closest()` function to use! You'll test out your semantic product search on a test query, computing a sorted list of the five most semantically similar products, based on the enriched data you gave the model.

Here's a reminder of the `find_n_closest()` function you created in the previous exercise:

The `create_embeddings()` function you created earlier is also available. Recall that it takes some text, and returns a list of lists containing the embeddings for each text. The `products` dictionary and the `product_embeddings` you created previously have also been loaded.

In [ ]:
# Create the query vector from query_text
query_text = "computer"
query_vector = create_embeddings(query_text)[0]

# Find the five closest distances
hits = find_n_closest(query_vector, product_embeddings, n=5)

print(f'Search results for "{query_text}"')
for hit in hits:
  # Extract the product at each index in hits
  product = products[hit['index']]

  print(product["title"])

### Section 2.2 - Recommendation system

#### Product recommendation system

In this exercise, you'll make a recommendation system for an online retailer that sells a variety of products. This system recommends three similar products to users who visit a product page but don't purchase, based on the last product they visited.

You've been provided with a list of dictionaries of products available on the site,

and a dictionary for the last product the user visited, stored in `last_product`.

The following custom functions defined earlier in the course are also available for you to use:

- `create_embeddings(texts)` → returns a list of embeddings for each text in `texts`.
- `create_product_text(product)` → combines the `product` features into a single string for embedding.
- `find_n_closest(query_vector, embeddings, n=3) `→ returns the `n` closest distances and their indexes between the `query_vector` and `embeddings`, based on cosine distances.

In [ ]:
last_product = {'title': 'Building Blocks Deluxe Set',
 'short_description': 'Unleash your creativity with this deluxe set of building blocks for endless fun.',
 'price': 34.99,
 'category': 'Toys',
 'features': ['Includes 500+ colorful building blocks',
  'Promotes STEM learning and creativity',
  'Compatible with other major brick brands',
  'Comes with a durable storage container',
  'Ideal for children ages 3 and up']}

In [ ]:
# Combine the features for last_product and each product in products
last_product_text = create_product_text(last_product)
product_texts = [create_product_text(product) for product in products]

# Embed last_product_text and product_texts
last_product_embeddings = create_embeddings(last_product_text)[0]
product_embeddings = create_embeddings(product_texts)

# Find the three smallest cosine distances and their indexes
hits = find_n_closest(last_product_embeddings, product_embeddings, n=3)

for hit in hits:
  product = products[hit['index']]
  print(product['title'])

#### Adding user history to the recommendation engine
For many recommendation cases, such as film or purchase recommendation, basing the next recommendation on one data point will be insufficient. In these cases, you'll need to embed all or some of the user's history for more accurate and relevant recommendations.

In this exercise, you'll extend your product recommendation system to consider all of the products the user has previously visited, which are stored in a list of dictionaries called `user_history`.

The following custom functions are available for you to use: `create_embeddings(texts)`, `create_product_text(product)`, and `find_n_closest(query_vector, embeddings, n=3)`. numpy has also been imported as `np`.

In [ ]:
user_history = [
 {'title': 'Remote-Controlled Dinosaur Toy',
  'short_description': 'Roar into action with this remote-controlled dinosaur toy with lifelike movements.',
  'price': 49.99,
  'category': 'Toys',
  'features': ['Realistic dinosaur sound effects',
   'Walks and roars like a real dinosaur',
   'Remote control included',
   'Educational and entertaining']},
 {'title': 'Building Blocks Deluxe Set',
  'short_description': 'Unleash your creativity with this deluxe set of building blocks for endless fun.',
  'price': 34.99,
  'category': 'Toys',
  'features': ['Includes 500+ colorful building blocks',
   'Promotes STEM learning and creativity',
   'Compatible with other major brick brands',
   'Comes with a durable storage container',
   'Ideal for children ages 3 and up']}]

In [ ]:
# Prepare and embed the user_history, and calculate the mean embeddings
history_texts = [create_product_text(article) for article in user_history]
history_embeddings = create_embeddings(history_texts)
mean_history_embeddings = np.mean(history_embeddings, axis=0)

# Filter products to remove any in user_history
products_filtered = [product for product in products if product not in user_history]

# Combine product features and embed the resulting texts
product_texts = [create_product_text(product) for product in products]
product_embeddings = create_embeddings(product_texts)

hits = find_n_closest(mean_history_embeddings, product_embeddings)

for hit in hits:
  product = products_filtered[hit['index']]
  print(product['title'])

### Section 2.3. - Embeddings for classification task

#### Embedding restaurant reviews

One common classification task that embeddings are great for is sentiment analysis. In this and the following exercises, you'll navigate through the workflow of performing sentiment analysis using embeddings.

You've been provided with a small sample of restaurant reviews, stored in `reviews`, and sentiment labels stored in `sentiments`:

In [ ]:
sentiments = [{'label': 'Positive'},
              {'label': 'Neutral'},
              {'label': 'Negative'}]

reviews = ["The food was delicious!",
           "The service was a bit slow but the food was good",
           "The food was cold, really disappointing!"]

You'll use zero-shot classification to classify the sentiment of these reviews by embedding the reviews and class labels.

The `create_embeddings()` function you created previously is also available to use.

In [ ]:
# Create a list of class descriptions from the sentiment labels
class_descriptions =  [sentiment['label'] for sentiment in sentiments]

# Embed the class_descriptions and reviews
class_embeddings = create_embeddings(class_descriptions)
review_embeddings = create_embeddings(reviews)

#### Classifying review sentiment
Now that you've calculated the embeddings, it's time to compute the cosine distances and extract the most similar label.

You'll do this by defining a function called `find_closest()`, which can be used to compare the embeddings between one vector and multiple others, and return the nearest distance and its index. You'll then loop over the reviews and and use `find_closest()` to find the closest distance for each review, extracting the classified label using the index.

The `class_embeddings` and `review_embeddings` objects you created in the last exercise are available for you to use, as well as the `reviews` and `sentiments`.

In [ ]:
# Define a function to return the minimum distance and its index
def find_closest(query_vector, embeddings):
  distances = []
  for index, embedding in enumerate(embeddings):
    dist = distance.cosine(query_vector, embedding)
    distances.append({"distance": dist, "index": index})
  return min(distances, key=lambda x: x["distance"])

for index, review in enumerate(reviews):
  # Find the closest distance and its index using find_closest()
  closest = find_closest(review_embeddings[index], class_embeddings)
  # Subset sentiments using the index from closest
  label = sentiments[closest['index']]['label']
  print(f'"{review}" was classified as {label}')

#### Embedding more detailed descriptions

One of the last predicted labels didn't seem representative of the review; this was probably down to the lack of information being captured when we're only embedding the class labels. This time, descriptions of each class will be embedded instead, so the model better "understands" that you're classifying restaurant reviews.

The following objects are available for you to use:

In [ ]:
sentiments = [{'label': 'Positive',
               'description': 'A positive restaurant review'},
              {'label': 'Neutral',
               'description':'A neutral restaurant review'},
              {'label': 'Negative',
               'description': 'A negative restaurant review'}]

reviews = ["The food was delicious!",
           "The service was a bit slow but the food was good",
           "The food was cold, really disappointing!"]

In [ ]:
# Extract and embed the descriptions from sentiments
class_descriptions = [x['description'] for x in sentiments]
class_embeddings = create_embeddings(class_descriptions)
review_embeddings = create_embeddings(reviews)

def find_closest(query_vector, embeddings):
  distances = []
  for index, embedding in enumerate(embeddings):
    dist = distance.cosine(query_vector, embedding)
    distances.append({"distance": dist, "index": index})
  return min(distances, key=lambda x: x["distance"])

for index, review in enumerate(reviews):
  closest = find_closest(review_embeddings[index], class_embeddings)
  label = sentiments[closest['index']]['label']
  print(f'"{review}" was classified as {label}')

## Chapter 3 - Vector databases

### Section 3.1 - Vector databases for embedding systems

### Section 3.2 - Creating vector database with ChromaDB

#### Getting started with ChromaDB

In the following exercises, you'll use a vector database to embed and query 1000 films and TV shows from the Netflix dataset introduced in the video. The goal will be to use this data to generate recommendations based on a search query. To get started, you'll create the database and collection to store the data.

`chromadb` is available for you to use, and the `OpenAIEmbeddingFunction()` has been imported from `chromadb.utils.embedding_functions`. As with the first two chapters, you don't need to provide an OpenAI API key in this chapter.

In [ ]:
import chromadb
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
from openai import OpenAI

client = OpenAI(api_key= api_key)

In [ ]:
# Create a persistant client
client = chromadb.PersistentClient(path="vector_db/")

# Create a netflix_title collection using the OpenAI Embedding function
collection = client.create_collection(
    name="netflix_titles",
    embedding_function=OpenAIEmbeddingFunction(model_name="text-embedding-3-small", api_key="<OPENAI_API_TOKEN>"),
    get_or_create=True
)

# List the collections
print(client.list_collections())

### Estimating embedding costs with tiktoken

Now that we've created a database and collection to store the Netflix films and TV shows, we can begin embedding data.

Before embedding a large dataset, it's important to do a cost estimate to ensure you don't go over any budget restraints. Because OpenAI models are priced by number of tokens inputted, we'll use OpenAI's `tiktoken` library to count the number of tokens and convert them into a dollar cost.

You've been provided with `documents`, which is a list containing all of the data to embed. You'll iterate over the list, encode each document, and count the total number of tokens. Finally, you'll use the model's pricing to convert this into a cost.

In [ ]:
import json

with open("data.json", "r", encoding="utf-8") as f:
    documents=json.load(f)

print(f"The dataset has {len(documents)} rows")

In [ ]:
import tiktoken

# Load the encoder for the OpenAI text-embedding-3-small model
enc = tiktoken.encoding_for_model("text-embedding-3-small")

# Encode each text in documents and calculate the total tokens
total_tokens = sum(len(enc.encode(text)) for text in documents)

cost_per_1k_tokens = 0.00002

# Display number of tokens and cost
print('Total tokens:', total_tokens)
print('Cost:', cost_per_1k_tokens * total_tokens/1000)

#### Adding data to the collection

Time to add those Netflix films and TV shows to your collection! You've been provided with a list of document IDs and texts, stored in `ids` and `documents`, respectively, which have been extracted from `netflix_titles.csv` using the following code:

As an example of what information will be embedded, here's the first document from documents:

All of the necessary functions and packages have been imported, and a persistent client has been created and assigned to client.

In [ ]:
ids = [("s"+str(i)) for i in range(1,1001)]

In [ ]:
# Recreate the netflix_titles collection
collection = client.create_collection(
  name="netflix_titles",
  embedding_function=OpenAIEmbeddingFunction(model_name="text-embedding-3-small", api_key=api_key),
  get_or_create=True  
)

In [ ]:
# Add the documents and IDs to the collection
collection.add(ids=ids, documents=documents)

In [ ]:
# Print the collection size and first ten items
print(f"No. of documents: {collection.count()}")
print(f"First ten documents: {collection.peek}")

In [ ]:
collection.peek().keys()

### Section 3.3 - Querying the Netflix collection

#### Querying the Netflix collection
Now that you've created and populated the `netflix_titles` collection, it's time to query it!

As a first trial, you'll use it to provide recommendations for films and TV shows about dogs to one of your colleagues who loves dogs!

The `netflix_titles` collection is still available to use, and `OpenAIEmbeddingFunction()` has been imported.

In [ ]:
# Retrieve the netflix_titles collection
collection = client.get_collection(
  name="netflix_titles",
  embedding_function=OpenAIEmbeddingFunction(model_name="text-embedding-3-small", api_key=api_key)
)

# Query the collection for "films about dogs"
result = collection.query(
  query_texts=["films about cats"],
  n_results=3
)

print(result)

#### Updating and deleting items from a collection
Just because the documents have been stored away in a vector database, that doesn't mean that you can't make changes to add to the collection or update existing items.

In this exercise, you've been provided with two new Netflix titles stored in `new_data`:

In [ ]:
new_data = [
    {"id": "s1001", "document": "Title: Cats & Dogs (Movie)\nDescription: A look at the top-secret, high-tech espionage war going on between cats and dogs, of which their human owners are blissfully unaware."},
    {"id": "s6884", "document": 'Title: Goosebumps 2: Haunted Halloween (Movie)\nDescription: Three teens spend their Halloween trying to stop a magical book, which brings characters from the "Goosebumps" novels to life.\nCategories: Children & Family Movies, Comedies'}]

You'll either add or update these IDs in the database depending on whether they're already present in the collection.

In [ ]:
collection = client.get_collection(
  name="netflix_titles",
  embedding_function=OpenAIEmbeddingFunction(model_name="text-embedding-3-small", 
                                             api_key=api_key)
)

# Update or add the new documents
collection.upsert(
    ids=[rec['id'] for rec in new_data],
    documents=[rec['document'] for rec in new_data]
)

# Delete the item with ID "s95"
collection.delete(ids=["s95"])

result = collection.query(
    query_texts=["films about dogs"],
    n_results=3
)
print(result)

### Section 3.4 - Multiple queries and filtering

In [ ]:
import csv

ids = []
metadata = []

In [ ]:
with open('./raw_data/netflix_titles_1000.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for i, row in enumerate(reader):
        ids.append("s"+str(i))
        metadata.append({
            "type": row['type'],
            "rating": row["rating"],
            "release_year": int(row["release_year"])})
        print(row)
        break
        

In [ ]:
collection.update(ids=ids, metadatas=metadata)

#### Querying with multiple texts
In many cases, you'll want to query the vector database using multiple query texts. Recall that these query texts are embedded using the same embedding function as when the documents were added.

In this exercise, you'll use the documents from two IDs in the `netflix_titles` collection to query the rest of the collection, returning the most similar results as recommendations.

The `netflix_titles` collection is still available to use, and `OpenAIEmbeddingFunction()` has been imported.

In [ ]:
collection = client.get_collection(
  name="netflix_titles",
  embedding_function=OpenAIEmbeddingFunction(model_name="text-embedding-3-small", api_key=api_key)
)

reference_ids = ['s999', 's1000']

# Retrieve the documents for the reference_ids
reference_texts = collection.get(ids=reference_ids)['documents']

# Query using reference_texts
result = collection.query(
  query_texts=reference_texts,
  n_results=3
)

print(result['documents'])

#### Filtering using metadata
Having metadata available to use in the database can unlock the ability to more easily filter results based on additional conditions. Imagine that the film recommendations you've be creating could access the user's set preferences and use those to further filter the results.

In this exercise, you'll be using additional metadata to filter your Netflix film recommendations. The netflix_titles collection has been updated to add metadatas to each `title`, including the `'rating'`, the age rating given to the title, and `'release_year'`, the year the title was initially released.

Here's a preview of an updated item:

In [ ]:
collection = client.get_collection(
  name="netflix_titles",
  embedding_function=OpenAIEmbeddingFunction(model_name="text-embedding-3-small", api_key=api_key)
)

reference_texts = ["children's story about a car", "lions"]

# Query two results using reference_texts
result = collection.query(
  query_texts=reference_texts,
  n_results=2,
  # Filter for titles with a G rating released before 2019
  where={
    "$and": [
        {"rating": 
        	{"$eq": "G"}
        },
        {"release_year": 
         	{"$lt": 2019}
        }
    ]
  }
)

print(result['documents'])